In [ ]:
# %pip install numpy matplotlib tifffile lxml

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tifffile import TiffFile, imread, imwrite
import json
from pprint import pprint
import xml.dom.minidom
import sciebo

Run the below cells to download datasets for the first two sections of the notebook

This might take a while. In the meantime, feel free to read a bit about the datasets. We won't do any analysis here.

1. `data_endoscope.tif` 1-photon microendoscopic data from mouse dorsal striatum [Reference](https://elifesciences.org/articles/28728#s3).
2. `Sue_2x_3000_40_-46.tif` (taken from CaImAn) dataset by Sue Koay and David Tank. 2-photon data from supragranular parietal cortex mouse during a virtual reality task.

In [ ]:
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/aLuGqYoZRFgwhzF', 'data/data_endoscope.tif')
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data/Sue_2x_3000_40_-46.tif')

## Reading TIFF File 

TIFF, which stands for Tagged Image File Format, is a flexible and adaptable file format for storing images and data within a single file. It can be used to store multiple frames as `pages`.

| Method | Description |
| ------ | ----------- |
| `imread('path/to/file')` | Reads TIFF files as NumPy arrays. |
| `TiffFile('path/to/file')` | Provides more detailed control over reading TIFF files, including accessing metadata and handling complex file structures. |
| `%%timeit` | Measures the execution time of code blocks, useful for performance optimization. |

**Example** Read `data_endoscope.tif`. How many frames does it have?

In [ ]:
frames = imread('data/data_endoscope.tif')
frames.shape

Read `Sue_2x_3000_40_-46.tif`. How many frames does this data have?

Read `Sue_2x_3000_40_-46.tif`. How long does it take?

Hint: Put `%%timeit` in the beginning of the cell

Read `Sue_2x_3000_40_-46.tif`. How long does it take to access the last frame?

Hint: Put `%%timeit` in the beginning of the cell

**Example** Let's see the plots. Plot the first frame

In [ ]:
plt.imshow(frames[0], cmap='gray')

Plot the last frame

**Example** Read only the first 3 frames from `Sue_2x_3000_40_-46.tif`

In [ ]:
frames = imread('data/Sue_2x_3000_40_-46.tif', key=(0, 1, 2))
frames.shape

Read only 1, 3, 5, 7, 9th frame from `Sue_2x_3000_40_-46.tif`

Read only first 100 frames from `Sue_2x_3000_40_-46.tif`

Hint: use range(0, 100)

**Example** Read `data_endoscope.tif` with `TiffFile`?

In [ ]:
f = TiffFile('data/data_endoscope.tif')
frames = np.array([page.asarray() for page in f.pages])

Read `Sue_2x_3000_40_-46.tif` with TiffFile

How long does it take to read `Sue_2x_3000_40_-46.tif` with TiffFile?

How long does it take to access only the last frame with TiffFile?

## Metadata handling and TIFF file writing 

This section focuses on metadata handling and writing TIFF files with custom metadata, for managing and documenting image datasets in research. The methods demonstrated include extracting metadata from TIFF files, writing TIFF files with specified photometric interpretations, and embedding custom JSON-formatted metadata into TIFF files. 

| Method | Description |
| ------ | ----------- |
| `TiffFile('path/to/file')` | Opens a TIFF file for reading, providing access to image data and metadata. |
| `.pages[N].tags` | Accesses the tags (metadata) of the Nth page in the TIFF file. |
| `imwrite()` | Writes image data to a TIFF file, with options for specifying photometric interpretation and embedding custom metadata. |
| `json.dumps()` | Converts a Python dictionary into a JSON-formatted string suitable for embedding as metadata. |

**Example** Read and print basic information about `data_endoscope.tif`

In [ ]:
f = TiffFile('data/data_endoscope.tif')
print(f)

Read and print basic information about Sue_2x_3000_40_-46.tif

**Example** Print detailed information of first page of `data_endoscope.tif`

In [ ]:
f = TiffFile('data/data_endoscope.tif')
print(f.pages[0].tags)

Print detailed information of 500th page of `data_endoscope.tif`. How is it same or different from that of the first page?

Print detailed information of 1000th page of `data_endoscope.tif`. How is it same or different from that of the other two pages?

**Example** Extract the value of 'ImageWidth' on 1000th page of `data_endoscope`

In [ ]:
f = TiffFile('data/data_endoscope.tif')
print(f.pages[999].tags['ImageWidth'].value)

Extract the value of 'PhotometricInterpretation' on 1000th page of `data_endoscope`

Extract the value of 'ResolutionUnit' on 1000th page of `data_endoscope`

Let's practice writing TIFF files. Run the below cell to make three 100x100 frames/pages of random values

In [ ]:
frame1 = np.random.rand(100, 100)  # Frame 1
frame2 = np.random.rand(100, 100)  # Frame 2
frame3 = np.random.rand(100, 100)  # Frame 3

**Example** Write frame1, frame2, frame3 as three pages of output_file.tiff with `minisblack` photometric interpretation. Read the tags. What is 'minisblack' photometric interpretation?

In [ ]:
tiff_path = 'output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], photometric='minisblack')

f = TiffFile('output_file.tiff')
print(f.pages[-1].tags)

Write frame1, frame2, frame3 as three pages of output_file.tiff with `rgb` photometric interpretation. Read the tags. What is 'rgb' photometric interpretation?

In [ ]:
tiff_path = 'output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], photometric='rgb')

f = TiffFile('output_file.tiff')
print(f.pages[-1].tags['PhotometricInterpretation'].value)

**Example** Write `Experiment` as `Calcium Imaging` into metadata

In [ ]:
metadata = {
    'Experiment': 'Calcium Imaging',
}
metadata_json = json.dumps(metadata)
tiff_path = 'output_file.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], description=metadata_json)

f = TiffFile('output_file.tiff')
print(f.pages[0].tags['ImageDescription'].value)

Write `Experiment` as `Calcium Imaging` and `Date` to today's date into metadata

Write `Experiment` as `Calcium Imaging`, `Date` to today's date, and `Researcher` to your name into metadata

## OME data model and structure

This section introduces the OME-TIFF format, a specialized version of TIFF for microscopy images, which includes extensive metadata in the OME-XML standard format. This metadata can describe the image dimensions, acquisition parameters, and much (much) more, making OME-TIFF an important format for scientific imaging.

| Method | Description |
| ------ | ----------- |
| `TiffFile()` | Opens an OME-TIFF file for reading both image data and metadata. |
| `ome_metadata` | Accesses the embedded OME-XML metadata from an OME-TIFF file. |
| `xml.dom.minidom.parseString()` | Parses the OME-XML string and pretty-prints it, enhancing readability. |
| `plt.imshow()` | Visualizes individual image frames, useful for inspecting the data. |
| `FuncAnimation` | Creates an animation from a sequence of frames, ideal for visualizing changes over time or space in the dataset. |

In [ ]:
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/IQzFzXXTosmY2MY', 'data/output.ome.tiff')
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/QNOpF1JvRPM0qWm', 'data/multifile-Z1.ome.tiff')
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/8sKghg9SGBttZW5', 'data/multifile-Z2.ome.tiff')
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/6xtxbSysECFPrHy', 'data/multifile-Z3.ome.tiff')
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/sxwwSUGmtQtSZ0l', 'data/multifile-Z4.ome.tiff')
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/NRgPnsz3Pnhkv3r', 'data/multifile-Z5.ome.tiff')


**Example** Read and print OME metadata from output_file.tiff. Is there any?

In [ ]:
f = TiffFile('output_file.tiff')
ome_metadata = f.ome_metadata
print(ome_metadata)

Read and print ome metadata of `output.ome.tiff`. Is it readable?

Let's make it more readable. Instead of `print` use `pprint` which means `pretty print`

Better. The format of the output is XML. XML, which stands for eXtensible Markup Language, is a markup language that defines a set of rules for encoding documents in a format that is both human-readable and machine-readable. 

**Example** Read ome metadata of output.ome.tiff and use inbuilt library `xml` to parse the string format to pretty xml. 

In [ ]:
f = TiffFile('data/output.ome.tiff')
ome_metadata = f.ome_metadata
dom = xml.dom.minidom.parseString(ome_metadata) 
pprint(dom.toprettyxml())

Read ome metadata of multifile-Z1.ome.tiff and use inbuilt library `xml` to parse the string format to pretty xml 

Let's download another ome file for a fun visualization exercise. Run the below cell.

This might take a while, so feel free to read more about the dataset: https://downloads.openmicroscopy.org/images/OME-TIFF/2016-06/MitoCheck/

In [ ]:
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/jui4quIkGdkENPb', 'data/00001_01.ome.tiff')

Let's use `00001_01.ome.tiff` for the rest of the exercises. Read its metadata and pretty print in pretty xml.

Looking through its data, what are the X, Y, and temporal dimensions?

Confirm the dimensions by checking shape of the image data

**Example** It's been a while since we visualized something! Let's see how the first frame looks like? What kind of data is this?

In [ ]:
f = TiffFile('data/00001_01.ome.tiff')
image_data = f.asarray()
plt.imshow(image_data[0]);

Let's see how the second frame looks like.

Let's see how the last frame looks like.

In [ ]:
from matplotlib.animation import FuncAnimation

frames = image_data.copy()

fig, ax = plt.subplots()
frame = frames[0]  
im = ax.imshow(frame, animated=True)

def update(frame):
    im.set_array(frame)
    return [im]

ani = FuncAnimation(fig, update, frames=frames, blit=True, interval=50);

from IPython.display import HTML
HTML(ani.to_jshtml())


## Writing OME Data and metadata

In [ ]:
frame1 = np.random.rand(100, 100)  # Frame 1

Write extra metadata 'Experiment' as 'Calcium Imaging'

In [ ]:
metadata = {
    'Experiment': 'Calcium Imaging',
}
tiff_path = 'output.ome.tiff'
imwrite(tiff_path, [frame1, frame2, frame3], metadata={'Description': str(metadata)})

f = TiffFile('output.ome.tiff')
ome_metadata = f.ome_metadata
pprint(ome_metadata)

## Open Exploration

OME metadata is (usually) automatically generated by software used for data acquisition. Typically, the custom metadata is not added.

Is there any other software that can help with tiff and ome-tiff metadata? YES!

1. Download [ImageJ](https://imagej.net/ij/download.html). 
2. Download [Bioformatter plugin](https://downloads.openmicroscopy.org/bio-formats/7.2.0/artifacts/?C=M;O=D). The file you have to download `bioformats_package.jar`
3. Place the `bioformats_package.jar` in the `plugins` folder of `ImageJ`

**Use Bio-Format Importer from Plugins tab in ImageJ to import `00001_01.ome.tiff`. It might take a while to laod. Check `open xml`. Feel free to explore other options** 


For more information on how to open: [Small tutorial](https://docs.openmicroscopy.org/bio-formats/5.8.2/users/imagej/installing.html)

